# Wave Phase Dynamics v3: Spectra from Activations

What’s new:
- Extract FFT + Hilbert spectra **from model activations** per layer/head
- Keep inverted Zipf prior for comparison, but focus on hidden-state spectra
- Use more interesting prompts (technical, legal, riddle) for visuals


In [ ]:
# Imports and setup
import math
import json
from pathlib import Path
from typing import Dict, List, Tuple
from dataclasses import dataclass
from collections import Counter

import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib import cm
from scipy.signal import hilbert
from scipy.fft import fft, fftfreq

from transformers import GPT2LMHeadModel, GPT2Tokenizer

try:
    from datasets import load_dataset
    HAS_DATASETS = True
except ImportError:
    HAS_DATASETS = False
    print("datasets not available, using fallback word frequencies")

NOTEBOOK_DIR = Path.cwd()
FIG_DIR = NOTEBOOK_DIR / "figs_wave_v3"
FIG_DIR.mkdir(exist_ok=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


In [ ]:
# Load GPT-2
model_name = "gpt2"
print(f"Loading {model_name}...")
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name, output_attentions=True)
model.to(device)
model.eval()

n_layers = model.config.n_layer
n_heads = model.config.n_head
d_model = model.config.n_embd
print(f"Model: {n_layers} layers, {n_heads} heads, d_model={d_model}")


## Inverted Zipf wave encoder (kept for comparison)


In [ ]:
# Build word frequency table
def build_word_frequency_table(tokenizer, max_words: int = 50000) -> Dict[str, int]:
    if HAS_DATASETS:
        print("Loading wikitext-2 for frequency estimation...")
        try:
            dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train")
            word_counts = Counter()
            for example in dataset:
                text = example["text"]
                if text.strip():
                    word_counts.update(text.lower().split())
            ranked = word_counts.most_common(max_words)
            word_rank = {word: rank + 1 for rank, (word, _) in enumerate(ranked)}
            print(f"Built frequency table with {len(word_rank)} words from wikitext-2")
            return word_rank
        except Exception as e:
            print(f"Failed to load wikitext-2: {e}")
    
    print("Using GPT-2 vocab order as frequency proxy...")
    word_rank = {}
    for token_id in range(tokenizer.vocab_size):
        token = tokenizer.decode([token_id]).strip().lower()
        if token and token not in word_rank:
            word_rank[token] = len(word_rank) + 1
    print(f"Built frequency table with {len(word_rank)} tokens")
    return word_rank

word_freq_table = build_word_frequency_table(tokenizer)
common = sorted(word_freq_table.items(), key=lambda x: x[1])[:5]
rare = sorted(word_freq_table.items(), key=lambda x: x[1])[-5:]
print(f"Common examples: {common}")
print(f"Rare examples: {rare}")


In [ ]:
# Inverted Zipf wave encoder (for comparison)
@dataclass
class WaveConfig:
    freq_min: float = 0.5
    freq_max: float = 10.0
    n_harmonics: int = 4
    sample_rate: int = 100
    duration: float = 2.0

class ZipfWaveEncoder:
    def __init__(self, tokenizer, word_freq_table: Dict[str, int], config: WaveConfig):
        self.tokenizer = tokenizer
        self.word_freq_table = word_freq_table
        self.config = config
        self.max_rank = max(word_freq_table.values()) if word_freq_table else 50000
        n_samples = int(config.sample_rate * config.duration)
        self.t = np.linspace(0, config.duration, n_samples)
    
    def token_to_frequency(self, token: str) -> float:
        rank = self.word_freq_table.get(token.strip().lower(), self.max_rank)
        log_rank = np.log(rank + 1)
        log_max = np.log(self.max_rank + 1)
        normalized = 1.0 - (log_rank / log_max)  # inverted: common -> high freq
        return self.config.freq_min + (self.config.freq_max - self.config.freq_min) * normalized
    
    def token_to_wave(self, token: str, phase_offset: float = 0.0) -> np.ndarray:
        freq = self.token_to_frequency(token)
        wave = np.zeros_like(self.t, dtype=np.complex128)
        for h in range(1, self.config.n_harmonics + 1):
            amplitude = 1.0 / h
            wave += amplitude * np.exp(1j * 2 * np.pi * freq * h * self.t + 1j * phase_offset * h)
        return wave
    
    def encode_sequence(self, text: str) -> Tuple[List[str], np.ndarray, np.ndarray]:
        token_ids = tokenizer.encode(text)
        tokens = [tokenizer.decode([tid]) for tid in token_ids]
        freqs = np.array([self.token_to_frequency(t) for t in tokens])
        waves = np.zeros((len(tokens), len(self.t)), dtype=np.complex128)
        for i, token in enumerate(tokens):
            phase_offset = 2 * np.pi * i / len(tokens)
            waves[i] = self.token_to_wave(token, phase_offset)
        return tokens, freqs, waves

wave_config = WaveConfig()
wave_encoder = ZipfWaveEncoder(tokenizer, word_freq_table, wave_config)
print(f"Wave encoder (INVERTED): common -> {wave_config.freq_max} Hz, rare -> {wave_config.freq_min} Hz")


In [ ]:
# Inference helper
def run_inference(text: str) -> Dict:
    input_ids = tokenizer.encode(text, return_tensors="pt").to(device)
    tokens = [tokenizer.decode([tid]) for tid in input_ids[0]]
    with torch.no_grad():
        outputs = model(input_ids, output_attentions=True, output_hidden_states=True)
    return {
        "tokens": tokens,
        "input_ids": input_ids,
        "logits": outputs.logits.cpu(),
        "attentions": [a.cpu() for a in outputs.attentions],  # list len n_layers
        "hidden_states": [h.cpu() for h in outputs.hidden_states],  # len n_layers+1 (emb + layers)
        "n_layers": len(outputs.attentions),
        "n_heads": outputs.attentions[0].size(1),
    }

print("run_inference ready (returns attentions + hidden_states)")


In [ ]:
# Spectral analysis on activations (per head)

def head_context_from_hidden(result: Dict, layer_idx: int, head_idx: int, query_pos: int = -1) -> np.ndarray:
    """Approximate head context: attention weights over input hidden states to this layer.
    Uses hidden_states[layer_idx] as the input to layer `layer_idx`.
    This ignores the value projection but preserves attention weighting structure.
    """
    attn = result["attentions"][layer_idx][0, head_idx].numpy()  # [seq, seq]
    weights = attn[query_pos]  # [seq]
    hidden_in = result["hidden_states"][layer_idx][0].numpy()  # [seq, d_model]
    context = weights @ hidden_in  # [d_model]
    return context


def fft_spectrum(signal: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """FFT of a real signal -> freqs, magnitudes, phases (radians)."""
    fft_vals = fft(signal)
    freqs = fftfreq(len(signal), d=1.0)
    pos = freqs > 0
    return freqs[pos], np.abs(fft_vals[pos]), np.angle(fft_vals[pos])


def spectral_concentration_from_signal(signal: np.ndarray) -> float:
    freqs, mags, _ = fft_spectrum(signal)
    power = mags ** 2
    power = power / (power.sum() + 1e-10)
    entropy = -np.sum(power * np.log(power + 1e-10))
    max_entropy = np.log(len(power))
    return 1.0 - entropy / (max_entropy + 1e-10)


def hilbert_phase(signal: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    analytic = hilbert(signal)
    phase = np.unwrap(np.angle(analytic))
    envelope = np.abs(analytic)
    return phase, envelope


def head_phase_coherence(result: Dict, layer_idx: int, top_k: int = 3) -> float:
    """Coherence of dominant FFT phases across heads at a layer (query = last token)."""
    phases = []
    for h in range(result['n_heads']):
        ctx = head_context_from_hidden(result, layer_idx, h)
        _, mags, phs = fft_spectrum(ctx)
        if len(mags) == 0:
            continue
        top_idx = np.argsort(mags)[-top_k:]
        phases.extend(list(phs[top_idx]))
    if not phases:
        return 0.0
    return np.abs(np.mean(np.exp(1j * np.array(phases))))


def head_interference(result: Dict, layer_idx: int) -> float:
    """Interference ratio across heads: power of mean context vs sum of powers."""
    contexts = []
    for h in range(result['n_heads']):
        ctx = head_context_from_hidden(result, layer_idx, h)
        contexts.append(ctx)
    contexts = np.stack(contexts, axis=0)  # [heads, d_model]
    mean_ctx = contexts.mean(axis=0)
    power_mean = (mean_ctx ** 2).mean()
    power_sum = (contexts ** 2).mean()
    return power_mean / (power_sum + 1e-10)

print("Spectral helpers ready (FFT, Hilbert, per-head coherence/interference)")


## Prompts (more interesting cases)


In [ ]:
prompts = {
    "quantum": "The quantum mechanical wave function describes tunneling inside stars",
    "legal": "This agreement indemnifies the contractor against all third-party claims arising from negligence",
    "riddle": "I speak without a mouth and hear without ears; what am I?",
    "coding": "Write a Python function that returns the greatest common divisor using Euclid's algorithm",
    "ambiguous": "Describe the color of a square circle"
}
print(f"Loaded {len(prompts)} prompts")


In [ ]:
# Run spectral analysis on activations
analysis = {}

for name, prompt in prompts.items():
    print(f"Processing: {name} -> '{prompt[:60]}...'")
    result = run_inference(prompt)
    layer_coh = []
    layer_inter = []
    layer_spec = []
    for layer_idx in range(result['n_layers']):
        # per-head phase coherence
        layer_coh.append(head_phase_coherence(result, layer_idx, top_k=3))
        # interference across heads
        layer_inter.append(head_interference(result, layer_idx))
        # spectral concentration of mean context (heads averaged)
        contexts = []
        for h in range(result['n_heads']):
            ctx = head_context_from_hidden(result, layer_idx, h)
            contexts.append(ctx)
        mean_ctx = np.stack(contexts, axis=0).mean(axis=0)
        layer_spec.append(spectral_concentration_from_signal(mean_ctx))
    logits = result['logits'][0, -1, :]
    probs = F.softmax(logits, dim=-1)
    entropy = -(probs * probs.clamp(min=1e-10).log()).sum().item()
    top_token = tokenizer.decode([probs.argmax().item()])
    analysis[name] = {
        "prompt": prompt,
        "coherence": layer_coh,
        "interference": layer_inter,
        "spectral_concentration": layer_spec,
        "output_entropy": entropy,
        "top_token": top_token,
        "tokens": result['tokens']
    }

print("Done.")


## Plots: coherence, spectral concentration, interference (activations)


In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(12, 12))
prompt_names = list(analysis.keys())
colors = plt.cm.tab10(np.linspace(0, 1, len(prompt_names)))

# Coherence
ax = axes[0, 0]
for c, name in zip(colors, prompt_names):
    ax.plot(analysis[name]["coherence"], label=name, color=c)
ax.set_title("Head phase coherence across layers")
ax.set_xlabel("Layer"); ax.set_ylabel("Coherence R")
ax.grid(True, alpha=0.3); ax.legend()

# Spectral concentration
ax = axes[0, 1]
for c, name in zip(colors, prompt_names):
    ax.plot(analysis[name]["spectral_concentration"], label=name, color=c)
ax.set_title("Spectral concentration (mean context)")
ax.set_xlabel("Layer"); ax.set_ylabel("Concentration")
ax.grid(True, alpha=0.3)

# Interference
ax = axes[1, 0]
for c, name in zip(colors, prompt_names):
    ax.plot(analysis[name]["interference"], label=name, color=c)
ax.axhline(1.0, color='black', linestyle='--', alpha=0.5)
ax.set_title("Interference ratio across heads")
ax.set_xlabel("Layer"); ax.set_ylabel("Power ratio")
ax.grid(True, alpha=0.3)

# Output entropy vs final coherence
ax = axes[1, 1]
for c, name in zip(colors, prompt_names):
    ax.scatter(analysis[name]["output_entropy"], analysis[name]["coherence"][-1], color=c, label=name, s=80, edgecolors='black')
ax.set_xlabel("Output entropy")
ax.set_ylabel("Final-layer coherence")
ax.set_title("Confidence vs coherence")
ax.grid(True, alpha=0.3)
ax.legend()

# Table-like text
ax = axes[2, 0]
ax.axis('off')
rows = []
for name in prompt_names:
    rows.append(f"{name:12s} | top='{analysis[name]['top_token'].strip()[:12]:<12s}' | H={analysis[name]['output_entropy']:.2f}")
ax.text(0.0, 0.9, "\n".join(rows), fontsize=10, va='top')
ax.set_title("Top token / entropy")

# Empty placeholder for layout
axes[2, 1].axis('off')

plt.suptitle("Activation Spectral Metrics (interesting prompts)", fontsize=14)
plt.tight_layout()
plt.savefig(FIG_DIR / "v3_activation_metrics.png", dpi=150)
plt.show()


## Per-head spectra for an interesting prompt (quantum)


In [ ]:
target_name = "quantum"
prompt = prompts[target_name]
print(f"Per-head spectra for: {prompt}")
res = run_inference(prompt)

layer_to_show = [0, 5, 11]
fig, axes = plt.subplots(len(layer_to_show), 2, figsize=(12, 12))

for row, layer_idx in enumerate(layer_to_show):
    spectra = []
    for h in range(res['n_heads']):
        ctx = head_context_from_hidden(res, layer_idx, h)
        freqs, mags, phs = fft_spectrum(ctx)
        spectra.append(mags)
    # Pad to same length
    max_len = max(len(s) for s in spectra)
    spec_mat = np.zeros((res['n_heads'], max_len))
    for h, s in enumerate(spectra):
        spec_mat[h, :len(s)] = s
    ax = axes[row, 0]
    im = ax.imshow(spec_mat, aspect='auto', cmap='magma', origin='lower')
    ax.set_title(f"Layer {layer_idx} head spectra (mag)")
    ax.set_ylabel("Head")
    plt.colorbar(im, ax=ax, shrink=0.7)
    
    # Coherence / interference summary
    coh = head_phase_coherence(res, layer_idx)
    inter = head_interference(res, layer_idx)
    ax2 = axes[row, 1]
    ax2.bar(["coh", "inter"], [coh, inter], color=['teal', 'coral'])
    ax2.set_ylim(0, max(1.2, inter + 0.2))
    ax2.set_title(f"Layer {layer_idx}: coh={coh:.3f}, inter={inter:.3f}")
    ax2.grid(True, alpha=0.3)

plt.suptitle("Per-head FFT magnitude heatmaps (quantum prompt)", fontsize=14)
plt.tight_layout()
plt.savefig(FIG_DIR / "v3_quantum_head_spectra.png", dpi=150)
plt.show()


## Wave superposition (synthetic waves) for comparison


In [ ]:
prompt_wave = prompts["legal"]
print(f"Wave superposition prompt: {prompt_wave}")
tokens, freqs, waves = wave_encoder.encode_sequence(prompt_wave)
res_wave = run_inference(prompt_wave)

layers_to_show = [0, 5, 11]
fig, axes = plt.subplots(len(layers_to_show), 2, figsize=(14, 10))
coherences = []

for row, layer_idx in enumerate(layers_to_show):
    attn = res_wave["attentions"][layer_idx][0].mean(dim=0).numpy()
    query_attn = attn[-1, :]
    superposed = (query_attn[:, None] * waves).sum(axis=0)
    coh = np.abs(superposed / (np.abs(waves * query_attn[:, None]).sum(axis=0) + 1e-10)).mean()
    coherences.append(coh)
    ax1 = axes[row, 0]
    ax1.bar(range(len(tokens)), query_attn, color='steelblue')
    ax1.set_title(f"Layer {layer_idx} attention (query=last)")
    ax1.set_xticks(range(len(tokens)))
    ax1.set_xticklabels([t.strip()[:8] for t in tokens], rotation=45, ha='right')
    ax1.grid(True, alpha=0.3)
    ax2 = axes[row, 1]
    t_show = wave_encoder.t[:80]
    ax2.plot(t_show, superposed.real[:80], 'b-', linewidth=1.5)
    ax2.fill_between(t_show, -np.abs(superposed[:80]), np.abs(superposed[:80]), color='blue', alpha=0.2)
    ax2.set_title(f"Layer {layer_idx} superposition (coh~{coh:.3f})")
    ax2.grid(True, alpha=0.3)

axes[-1, 0].set_xlabel("Token")
axes[-1, 1].set_xlabel("Time (s)")
plt.suptitle("Synthetic wave superposition (legal prompt)", fontsize=14)
plt.tight_layout()
plt.savefig(FIG_DIR / "v3_wave_superposition_legal.png", dpi=150)
plt.show()
print("Layer coherences:", dict(zip(layers_to_show, coherences)))


In [ ]:
# Save analysis (JSON-safe)
def to_py(obj):
    if isinstance(obj, (np.floating, np.integer)):
        return obj.item()
    if isinstance(obj, (list, tuple)):
        return [to_py(x) for x in obj]
    if isinstance(obj, dict):
        return {k: to_py(v) for k, v in obj.items()}
    return obj

summary = {
    "model": model_name,
    "n_layers": n_layers,
    "n_heads": n_heads,
    "prompts": {
        name: {
            "prompt": data["prompt"],
            "coherence": [float(x) for x in data["coherence"]],
            "interference": [float(x) for x in data["interference"]],
            "spectral_concentration": [float(x) for x in data["spectral_concentration"]],
            "output_entropy": float(data["output_entropy"]),
            "top_token": data["top_token"],
            "tokens": data["tokens"],
        }
        for name, data in analysis.items()
    }
}
with open(FIG_DIR / "wave_phase_results_v3.json", "w") as f:
    json.dump(to_py(summary), f, indent=2)
print(f"Saved to {FIG_DIR / 'wave_phase_results_v3.json'}")


## Summary
- Added activation-based spectra: FFT + Hilbert on head-specific contexts (using attention weights over layer inputs)
- Metrics: head phase coherence, head interference, spectral concentration of mean context
- More interesting prompts (quantum, legal, riddle, coding, ambiguous) for clearer visuals
- Kept synthetic inverted-Zipf waves only as a side-by-side intuition check

Key figures (saved to `figs_wave_v3/`):
- `v3_activation_metrics.png`: layer curves for coherence, spectral concentration, interference
- `v3_quantum_head_spectra.png`: per-head FFT heatmaps on the quantum prompt
- `v3_wave_superposition_legal.png`: synthetic wave superposition on a legal prompt
